In [4]:
from lxml import etree as ET
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib
import multiprocessing
#from multiprocessing import Process
#print("Number of cpu : ", multiprocessing.cpu_count())



In [5]:
%load_ext autoreload
%autoreload 2
config = RClib.loadconfigs('.\config.json')
scandf3 = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
scandf3= scandf3.apply(RClib.defineResultOutput, foldername='04_GEO', axis=1)
#while 'reconstruct-rccmdpath'.isin(scandf3['processingstate'])] and 'model-rccmdpath'.notin(scandf3['processingstate']) :
    #tobemodeleddf = scandf3['reconstruct-rccmdpath'.isin(scandf3['processingstate']) and 'model-rccmdpath'.notin(scandf3['processingstate']) ]
scandf4 = pd.DataFrame()
for index,scan in scandf3.iterrows():
    scan['rcboxpath'] = Path(scan['RCoutputfolder'] / 'base.rcbox')
    if 'align-rccmdpath' in scan['processingstate'] and scan['rcboxpath'].is_file():
        if not 'commandlist_reconstructsettings' in scan['processingstate']:
            scan['commandlist_reconstructsettings']=Path(config['reconstructsettings'])
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_reconstructsettings')
        if not 'reconstruct-rccmdpath' in scan['processingstate'] :
            scan['commandlist_reconstruct'] = []
            scan['commandlist_reconstruct'].append('-setReconstructionRegion ' + str(scan['rcboxpath']))  
            scan['commandlist_reconstruct'].append('-calculateNormalModel')  
            scan['commandlist_reconstruct'].append('-renameSelectedModel ' + scan['id'] + '_highpoly')    
            scan['commandlist_reconstruct'].append('-save')
            scan['commandlist_reconstruct'].append('-quit')  
            scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_reconstruct', rccmdpathfield='reconstruct-rccmdpath')
            print('Starting reconstruction for  ', scan['id'])
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='reconstruct-rccmdpath' )
    scandf4 = scandf4.append(scan)
#pd.write_csv(Path(config['workspace']) / 'rcprocessingdf.csv', sep=';')
scandf4.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
scandf4 = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
#while 'reconstruct-rccmdpath'.isin(scandf3['processingstate'])] and 'model-rccmdpath'.notin(scandf3['processingstate']) :
    #tobemodeleddf = scandf3['reconstruct-rccmdpath'.isin(scandf3['processingstate']) and 'model-rccmdpath'.notin(scandf3['processingstate']) ]
scandf5 = pd.DataFrame()
for index,scan in scandf4.iterrows():
    if 'reconstruct-rccmdpath' in scan['processingstate']:
        if not 'commandlist_texturingsettings' in scan['processingstate']:
            scan['commandlist_texturingsettings']=Path(config['texturingsettings'])
            print('Texturing settings set for ', scan['id'])
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_texturingsettings')
        if not 'texturing-rccmdpath' in scan['processingstate'] :
            scan['commandlist_texturing'] = []
            scan['commandlist_texturing'].append('-selectModel ' + scan['id'] + '_highpoly') 
            scan['commandlist_texturing'].append('-smooth') 
            scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_highpoly' + '_smoothed')
            scan['commandlist_texturing'].append('-cleanModel')
            scan['commandlist_texturing'].append('-correctColors geometry') 
            scan['commandlist_texturing'].append('-unwrap')  
            scan['commandlist_texturing'].append('-calculateTexture')
            scan['commandlist_texturing'].append('-simplify 10000000')
            scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_10M')
            scan['commandlist_texturing'].append('-cleanModel')
            scan['modelspaths'] = {'modelpath_10M': scan['Resultoutputfolder'] / Path(config['exportname_prefix'] + scan['id'] + '_10M.obj')}
            print(scan['modelspaths'])
            scan['commandlist_texturing'].append('-exportModel ' + scan['id'] + '_10M ' + str(scan['modelspaths'].get('modelpath_10M')) + ' ' + config['exportsettings1'] )
            scan['commandlist_texturing'].append('-selectModel ' + scan['id'] + '_highpoly' + '_smoothed')   
            scan['commandlist_texturing'].append('-simplify 40000000')
            scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_40M')
            scan['commandlist_texturing'].append('-cleanModel') 
            scan['modelspaths']['modelpath_40M'] = scan['Resultoutputfolder'] / Path( config['exportname_prefix'] + scan['id'] + '_40M.obj')
            scan['commandlist_texturing'].append('-exportModel ' + scan['id'] + '_40M ' + str(scan['modelspaths'].get('modelpath_40M')) + ' ' + config['exportsettings1'] )           
            scan['commandlist_texturing'].append('-save')
            scan['commandlist_texturing'].append('-quit')  
            scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_texturing', rccmdpathfield='texturing-rccmdpath')
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='texturing-rccmdpath' )
    scandf5 = scandf5.append(scan)
scandf5.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))


Texturing settings set for  PG_boat91_test
{'modelpath_10M': WindowsPath('E:/RCprocessing_test/PG_boat91_test/04_GEO/Zed-ext_0000001_PG_boat91_test_10M.obj')}


TypeError: can only concatenate str (not "WindowsPath") to str